## Installs and Imports

In [3]:
import os
import tempfile
import base64
import json

import numpy as np
import matplotlib.pyplot as plt

import torch

import tarfile
import urllib.request

from itkwidgets import view
from ipywidgets import interact

from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, command, Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment, JobService, Data, CodeConfiguration, OnlineRequestSettings, AmlCompute
from azure.core.exceptions import ResourceNotFoundError

from monai.apps import DecathlonDataset
from monai.data import DataLoader, Dataset
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, EnsureTyped, Orientationd, Spacingd, NormalizeIntensityd, MapTransform
from monai.visualize.utils import blend_images

## Define central variables

In [4]:
# AzureML Workspace
subscription_id = 'b7d41fc8-d35d-41db-92ed-1f7f1d32d4d9'
resource_group = 'ams-monai-seg-reg-rg'
workspace = 'ams-monai-seg-reg-wg'

# Training
experiment = 'brain-tumor-segmentation' # AzureML experiment name
dataset_name="BRATS2021"
#train_target = 'NC96adsA100'
#train_target = 'NC64asT4v3'
train_target = 'NC96A100'

# Deployment
online_endpoint_name = "brain-tumor-SegResNet-1"
registered_model_name = 'SegRes-1'
deployment_name = 'red'

# Visualization and validation sample
sample_image = './samples-2021/BraTS2021_00402/BraTS2021_00402_flair.nii.gz' # pick flair modality
sample_image_t1 = './samples-2021/BraTS2021_00402/BraTS2021_00402_t1.nii.gz'
sample_image_t1ce = './samples-2021/BraTS2021_00402/BraTS2021_00402_t1ce.nii.gz'
sample_image_t2 = './samples-2021/BraTS2021_00402/BraTS2021_00402_t2.nii.gz'
sample_label = './samples-2021/BraTS2021_00402/BraTS2021_00402_seg.nii.gz'

In [5]:
# Connect to AzureML Workspace
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

2023-03-20 20:46:09,911 - No environment configuration found.
2023-03-20 20:46:09,913 - ManagedIdentityCredential will use Azure ML managed identity


## Submit Parallel Training Job
We are using multi-GPU PyTorch Distrubuted Data Parallel training with scalable Azure ML compute resources. Feel free to change the number of cluster nodes `instance_count` and the number of GPUs per node `process_count_per_instance` to leverage depending on the compute SKU you provisioned.  
Note that you can interact with the job for monitoring or debugging using JupyterLab, VSCode, or Tensorboard during training.

## Deploy model to a Managed Endpoint

In [6]:
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="MONAI 3d brain tumor segmentation req endpint",
    auth_mode="key",
    tags={
        "training_dataset": "Medical Segmentation Decathlon: Brain tumor segmentation",
        "model_type": "pytorch",
        "dataset" : dataset_name,
    },
)

endpoint = ml_client.begin_create_or_update(endpoint)

2023-03-20 20:47:19,692 - DefaultAzureCredential acquired a token from ManagedIdentityCredential
2023-03-20 20:47:20,604 - DefaultAzureCredential acquired a token from ManagedIdentityCredential


In [10]:
endpoint = ml_client.online_endpoints.get(online_endpoint_name)
print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

Endpoint brain-tumor-segresnet-1 provisioning state: Succeeded


In [11]:
# Let's pick the latest version of the model
latest_model_version = max([int(m.version) for m in ml_client.models.list(name= registered_model_name)])

print(f'Latest version of {registered_model_name} found: {latest_model_version}')

2023-03-20 20:49:04,559 - DefaultAzureCredential acquired a token from ManagedIdentityCredential
Latest version of SegRes-1 found: 1


In [12]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version= latest_model_version)

In [33]:
from azureml.core.model import Model
from azureml.core.workspace import Workspace

ws = Workspace(subscription_id, resource_group, workspace)

#Model.list(get_model_path("best_metric_model", version=None, _workspace=None))
Model.list(ws)

2023-03-20 22:22:47,863 - Created a worker pool for first use


[Model(workspace=Workspace.create(name='ams-monai-seg-reg-wg', subscription_id='b7d41fc8-d35d-41db-92ed-1f7f1d32d4d9', resource_group='ams-monai-seg-reg-rg'), name=SegRes-1, id=SegRes-1:1, version=1, tags={}, properties={'azureml.datastoreId': '/subscriptions/b7d41fc8-d35d-41db-92ed-1f7f1d32d4d9/resourceGroups/ams-monai-seg-reg-rg/providers/Microsoft.MachineLearningServices/workspaces/ams-monai-seg-reg-wg/datastores/workspaceartifactstore'}),
 Model(workspace=Workspace.create(name='ams-monai-seg-reg-wg', subscription_id='b7d41fc8-d35d-41db-92ed-1f7f1d32d4d9', resource_group='ams-monai-seg-reg-rg'), name=brat-best-model, id=brat-best-model:1, version=1, tags={}, properties={'azureml.datastoreId': '/subscriptions/b7d41fc8-d35d-41db-92ed-1f7f1d32d4d9/resourceGroups/ams-monai-seg-reg-rg/providers/Microsoft.MachineLearningServices/workspaces/ams-monai-seg-reg-wg/datastores/workspaceartifactstore'}),
 Model(workspace=Workspace.create(name='ams-monai-seg-reg-wg', subscription_id='b7d41fc8-d35

In [44]:
score_env = "brats-inference-env"

model = ml_client.models.get(name=registered_model_name, version= latest_model_version)

model.path

os.getenv("AZUREML_MODEL_DIR")



In [45]:
# create an online deployment.
deployment = ManagedOnlineDeployment(
    name = deployment_name,
    endpoint_name = online_endpoint_name,
    model = model,
    environment = score_env + "@latest",
    code_configuration=CodeConfiguration(code= "./src", scoring_script="score-brats21.py"),
    instance_type = "Standard_NC4as_T4_v3",
    instance_count = 1,
    request_settings= OnlineRequestSettings(request_timeout_ms = 90000),
    
)
deployment = ml_client.begin_create_or_update(deployment)

Exception: [31m

Error: 

1) YAML file cannot be parsed.


Details: 

Failed to submit deployment red due to syntax errors in scoring script score-brats21.py.
Error on line 45:      model_path = os.path.join(

If you wish to bypass this validation use --skip-script-validation paramater.

Resolutions:
Double-check your YAML file for syntax and formatting errors.
If using the CLI, you can also check the full log in debug mode for more details by adding --debug to the end of your command
Additional Resources: The easiest way to author a yaml specification file is using IntelliSense and auto-completion Azure ML VS code extension provides: https://code.visualstudio.com/docs/datascience/azure-machine-learning. To set up VS Code, visit https://docs.microsoft.com/azure/machine-learning/how-to-setup-vs-code
[39m

In [16]:
# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{}
https://brain-tumor-segresnet-1.westeurope.inference.ml.azure.com/score


In [18]:
os.getenv("AZUREML_MODEL_DIR")


## Get Predictions

In [ ]:
# Encode input images for JSON request file
with open(sample_image, "rb") as image_file:
    flair_encoded = base64.b64encode(image_file.read()).decode('utf-8')

with open(sample_image_t1, "rb") as image_file:
    t1_encoded = base64.b64encode(image_file.read()).decode('utf-8')

with open(sample_image_t1ce, "rb") as image_file:
    t1ce_encoded = base64.b64encode(image_file.read()).decode('utf-8')

with open(sample_image_t2, "rb") as image_file:
    t2_encoded = base64.b64encode(image_file.read()).decode('utf-8')

request_data = {
    "data": [{"flair": flair_encoded, "t1": t1_encoded, 
              "t1ce": t1ce_encoded, "t2": t2_encoded
             }]
}

# Write the JSON request data to a file
with open("request-brats2021_1.json", "w") as outfile:
    json.dump(request_data, outfile)

In [ ]:
# Send request to Managed Online Endpoint
response = ml_client.online_endpoints.invoke(
    endpoint_name= online_endpoint_name,
    deployment_name= deployment_name,
    request_file="./request-brats2021_1.json",
)


In [ ]:

# convert response to numpy array with dimensions channel, height, width, slice
json_response = json.loads(response)
pred_vol = np.array(json_response)

## Review Predictions
We are inspecting the predictions for the tumor core segmentations and compare them with the ground truth annotations.

In [ ]:
class ConvertToMultiChannelBasedOnBratsClassesd(MapTransform):
    """
    Convert labels to multi channels based on brats 2021 classes:
    label 1 necrotic tumor core (NCR)
    label 2 peritumoral edematous/invaded tissue 
    label 3 is not used in the new dataset version
    label 4 GD-enhancing tumor 
    The possible classes are:
      TC (Tumor core): merge labels 1 and 4
      WT (Whole tumor): merge labels 1,2 and 4
      ET (Enhancing tumor): label 4

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            # merge label 1 and label 4 to construct TC
            result.append(torch.logical_or(d[key] == 1, d[key] == 4))
            # merge labels 1, 2 and 4 to construct WT
            result.append(
                torch.logical_or(
                    torch.logical_or(d[key] == 1, d[key] == 2), d[key] == 4
                )
            )
            # label 4 is ET
            result.append(d[key] == 4)
            d[key] = torch.stack(result, axis=0).float()
        return d

val_transform = Compose(
[
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys="image"),
    EnsureTyped(keys=["image", "label"]),
    ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    Spacingd(
        keys=["image", "label"],
        pixdim=(1.0, 1.0, 1.0),
        mode=("bilinear", "nearest"),
    ),
    NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
])

data_list = [{'image': sample_image, 'label': sample_label}]
val_ds = Dataset(data=data_list, transform=val_transform)

img_vol = val_ds[0]["image"].numpy()
seg_vol = val_ds[0]["label"].numpy()


In [ ]:
def show_slice(slice_index=60):

    img = np.expand_dims(img_vol[0,:,:,slice_index], 0) # images
    true_seg = np.expand_dims(seg_vol[0,:,:,slice_index], 0) # annotated ground truth labels
    pred_seg = np.expand_dims(pred_vol[0,:,:,slice_index], 0)  # predicted labels
    
    blend = blend_images(img, true_seg, cmap='hsv')
    over_true = np.transpose(blend, (1,2,0))
    blend = blend_images(img, pred_seg, cmap='Blues')
    over_pred = np.transpose(blend, (1,2,0))
    
    fig, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(14, 7))

    ax1.imshow(over_true)
    ax1.set_title('Ground truth segmentations')
    ax2.imshow(over_pred)
    ax2.set_title('Predicted segmentations')
    
    plt.tight_layout()
    plt.show()

# Use the interact function to create a slider for the slice index
_ = interact(show_slice, slice_index=(0, img_vol.shape[-1]-1))